<a href="https://colab.research.google.com/github/airoprojects/visiope/blob/main/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
import torch

In [ ]:
!git clone https://github.com/sithu31296/semantic-segmentation
%cd semantic-segmentation
%pip install -e .

In [3]:
import sys
from google.colab import drive
drive.mount('/content/drive')

# On Colab the path to the module ti fixed once you have
# corretly set up the project with gitsetup.ipynb
fixed_path = '/content/drive/MyDrive/Github/visiope/dataloader/'
sys.path.insert(0, fixed_path)

# Insert here the path to the dataset on your drive
data_path = '/content/drive/MyDrive/Dataset/'

#to import trainer and loss
fixed_path_loss = '/content/drive/MyDrive/Github/visiope/loss/'
sys.path.insert(0, fixed_path_loss)

fixed_path_trainer = '/content/drive/MyDrive/Github/visiope/trainer/'
sys.path.insert(0,fixed_path_trainer )


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Mounted at /content/drive


In [4]:
from asloader import Ai4MarsImporter, Ai4MarsProcessor, Ai4MarsData

# sys.path.insert(1, '/content/drive/MyDrive/Github/visiope/loss')

# import custom_loss

## Show Available Pretrained Models

In [5]:
from semseg import show_models

show_models()

  No.  Model Names
-----  -------------
    1  SegFormer
    2  Lawin
    3  SFNet
    4  BiSeNetv1
    5  DDRNet
    6  FCHarDNet
    7  BiSeNetv2


## Load a Pretrained Model

Download a pretrained model's weights from the result table (ADE20K, CityScapes, ...) and put it in `checkpoints/pretrained/model_name/`.

In [ ]:
%pip install -U gdown

In [ ]:
import gdown
from pathlib import Path

ckpt = Path('./checkpoints/pretrained/segformer')
ckpt.mkdir(exist_ok=True, parents=True)

url = 'https://drive.google.com/uc?id=1-OmW3xRD3WAbJTzktPC-VMOF5WMsN8XT'
output = './checkpoints/pretrained/segformer/segformer.b3.ade.pth'

gdown.download(url, output, quiet=False)

In [6]:
from semseg.models import *

model = eval('SegFormer')(
    backbone='MiT-B3',
    num_classes=150
)

try:
    model.load_state_dict(torch.load('checkpoints/pretrained/segformer/segformer.b3.ade.pth', map_location='cpu'))
except:
    print("Download a pretrained model's weights from the result table.")
model.eval()

model.to(device)

print('Loaded Model')

Download a pretrained model's weights from the result table.
Loaded Model


In [7]:
!unzip /content/drive/MyDrive/Dataset/dataset.zip -d /content/drive/MyDrive/Dataset/

Archive:  /content/drive/MyDrive/Dataset/dataset.zip
replace /content/drive/MyDrive/Dataset/dataset/val.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [7]:
train_set = torch.load("/content/drive/MyDrive/Dataset/dataset/train.pt")
val_set = torch.load("/content/drive/MyDrive/Dataset/dataset/val.pt")
test_set = torch.load("/content/drive/MyDrive/Dataset/dataset/test.pt")

from torch.utils.data import DataLoader


#COLPA MIAA (MY FAULT)!!!!!!!
train_set.setDevice(device)
val_set.setDevice(device)
test_set.setDevice(device)




train_set.conversion('f')
test_set.conversion('f')
val_set.conversion('f')

train_set.set_grad()
test_set.set_grad()
val_set.set_grad()



train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=5, shuffle=True)
val_loader = DataLoader(val_set, batch_size=5, shuffle=True)

In [12]:
#control to delete
print(len(train_set))

image, label = train_loader.dataset.__getitem__(0)
print(image.requires_grad)

108
True


## Simple Image Inference

### Load Image

### Model Forward

### Postprocess

### Choose a Head

In [13]:
from torch.nn.functional import cross_entropy
#START TRAIN
# import train and loss
from astrainer import Ai4MarsTrainer
import asloss

loss_fn =  torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters())
training_set = train_loader
test_set =  test_loader
trainer = Ai4MarsTrainer(loss_fn, optimizer, training_set, test_set)




In [14]:
from pickle import NONE
image = 0
label = 0
for images, labels in train_loader:
  image = images
  label = labels
  break


prediction = model(image)
print(label.type())
label = label.type(torch.FloatTensor)
print(label.type())


print(prediction.shape)
new_prediction = torch.argmax(prediction, dim=1)
print(f"prediction BEFORE shape -> {new_prediction.shape}")

new_prediction = new_prediction[:,None, :, :]
print(f"prediction AFTER shape -> {new_prediction.shape}")
print(f"label shape -> {label.shape}")

print(new_prediction.type())
new_prediction = new_prediction.type(torch.FloatTensor)
print(label.type())

print(loss_fn(new_prediction, label))

torch.cuda.FloatTensor
torch.FloatTensor
torch.Size([64, 150, 64, 64])
prediction BEFORE shape -> torch.Size([64, 64, 64])
prediction AFTER shape -> torch.Size([64, 1, 64, 64])
label shape -> torch.Size([64, 1, 64, 64])
torch.cuda.LongTensor
torch.FloatTensor
tensor(-0.)


In [15]:
trainer.train_multiple_epoch(model,EPOCHS=1)


EPOCH: 1


RuntimeError: ignored